In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("check_inverse_kinematics.ipynb")

# Arm inverse kinematics

Do IK twice, once with gradient descent, once with a Jacobian.
All the code is in arm_ik_gradient.py and arm_ik_jacobian.py

Slides: https://docs.google.com/presentation/d/1dowamZVQqyLDZ-k-vWRV0rrQ-4dMVWhSCy25BV8a6bU/edit?usp=sharing

In [ ]:
# Doing the imports for you
import numpy as np
import json as json
import matplotlib.pyplot as plt

In [ ]:
import sys; 
sys.path.insert(0, '.')
sys.path.insert(0, '../Skills')

# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

In [ ]:
import arm_forward_kinematics as afk
import arm_ik_gradient as ik_gradient

## Gradient calculation, using distance of grasp point to target
There are tests for each step of this/each function to make sure that you're getting the correct (or same as the example code) values. 

In [ ]:
# Create the arm geometry
base_size_param = (1.0, 0.5)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)

arm_geometry = afk.create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

# Set some initial angles
angles_check = [0.0, np.pi/4, 0, [-np.pi/4.0, np.pi/4.0, -np.pi/4.0]]
afk.set_angles_of_arm_geometry(arm_geometry, angles_check)

In [ ]:
# First check - do you get the same answer for the gripper location?
gripper_loc = afk.get_gripper_location(arm_geometry)
exp_gripper_loc = (-0.3536, 1.4098)
if not np.isclose(gripper_loc[0], exp_gripper_loc[0], atol=0.01) or not np.isclose(gripper_loc[1], exp_gripper_loc[1], atol=0.01):
    print(f"Gripper loc, expected {exp_gripper_loc}, got {gripper_loc}")
else:
    print(f"Success: got same gripper location")


In [ ]:
# Set the target for the example here
target = (0.5, 1.5)

In [ ]:
# Plot so you can see what the arm angles produce and where the target is
fig, axs = plt.subplots(1, 1, figsize=(4, 4))

# Actually get the matrices
matrices = afk.get_matrices_all_links(arm_geometry)
afk.plot_complete_arm(axs, arm_geometry, matrices)

# Plot target too
axs.plot(target[0], target[1], '+r')
axs.set_title("Before gradient descent")

In [ ]:
# First check - vector to goal - see pic above, should point from green + to red one
vec = ik_gradient.vector_to_goal(arm_geometry, target)
if not vec.shape[0] == 2:
    print(f"Expected a 2x1 vector, got {vec.shape}")
elif not np.isclose(vec[0], 0.8535, atol=0.01) or not np.isclose(vec[1], 0.09019, atol=0.01):
    print(f"Expected (0.853, 0.090), got {vec}")
else:
    print("Passed vec check")

In [ ]:
# Second check - distance to goal, should be length of vector
dist_to_arm = ik_gradient.distance_to_goal(arm_geometry, target)
if not np.isclose(dist_to_arm, 0.858, atol=0.01):
    print(f"Expected 0.858, got {dist_to_arm:0.3}")
else:
    print(f"Passed distance check, dist {dist_to_arm:0.3}")

In [ ]:
# Third check - the gradient. 
grad = ik_gradient.calculate_gradient(arm_geometry, angles_check, target)
exp_grad = [0.9419, 0.4447, 0.2114, 0.0559]
if not len(grad) == 4:
    print(f"Expected a 4x1 list, got {grad}")
elif not np.all(np.isclose(grad, exp_grad, atol=0.01)):
    print(f"Expected {exp_grad} got {grad}")
else:
    print("Passed gradient check")

In [ ]:
grader.check("gradient_calculation")

## Gradient descent
This checks the actual gradient descent algorithm - does the gripper get closer to the goal at each time step? Does it eventually reach the goal?

Checks gradient_descent in arm_ik_gradient.py

_Type your answer here, replacing this text._

In [ ]:
# Main check - do we get out the new angles? Note, this assumes that you haven't changed up the step size
b_succ, angles_new, count = ik_gradient.gradient_descent(arm_geometry, angles_check, target, b_one_step=True)
ang_exp = [-0.942, 0.341, -0.211, [-0.841, 0, 0]]
if not len(angles_new) == 4:
    print(f"Expected {ang_exp}, got {angles_new}")
for a1, a2 in zip(angles_new, ang_exp):
    if not np.all(np.isclose(a1, a2, atol=0.01)):
        print(f"Expected {a1} got {a2}")
if not b_succ:
    print(f"Expected successful/improvement, got none")

In [ ]:
# This will actually change the arm geometry based on the new angles
afk.set_angles_of_arm_geometry(arm_geometry, angles_check)
last_dist = ik_gradient.distance_to_goal(arm_geometry, target)

# For first round of for loop
angles_new = angles_check
print(f"Starting distance {last_dist}")
for i in range(0, 3):
    b_succ, angles_new, count = ik_gradient.gradient_descent(arm_geometry, angles_new, target, b_one_step=True)
    afk.set_angles_of_arm_geometry(arm_geometry, angles_new)
    dist = ik_gradient.distance_to_goal(arm_geometry, target)
    if not b_succ:
        print(f"Expected successful/improvement after each iteration, got none")
    elif not dist < last_dist:
        print(f"Expected improvement, got none, dist {dist}")
    else:
        print(f"Passed iteration check {i}, dist {dist}")
    last_dist = dist

if not np.isclose(dist, 0.1, atol=0.01):
    print(f"Expected distance to be close to 0.1, got {dist}, count {count}")


In [ ]:
# Do gradient descent until gradient descent bottoms out
b_succ, angles_new, count = ik_gradient.gradient_descent(arm_geometry, angles_check, target, b_one_step=False)

# Set the new angles
afk.set_angles_of_arm_geometry(arm_geometry, angles_new)
dist = ik_gradient.distance_to_goal(arm_geometry, target)

if not b_succ:
    print(f"Expected successful/improvement, got none")
elif not np.isclose(dist, 0.063, atol=0.01):
    print(f"Expected distance to be close to 0.063, got {dist}, count {count}")
    
else:
    print(f"Passed full descent {dist:0.2}")

In [ ]:
# And plot the old and the new
# Plot so you can see what it should look like
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

for i, a in enumerate([angles_check, angles_new]):
    # Actually get the matrices
    afk.set_angles_of_arm_geometry(arm_geometry, a)
    matrices = afk.get_matrices_all_links(arm_geometry)
    afk.plot_complete_arm(axs[i], arm_geometry, matrices)

    # Plot target too
    axs[i].plot(target[0], target[1], '+r')

axs[0].set_title("Before gradient descent")
axs[1].set_title(f"After gradient descent count {count} dist {dist:0.3}")

In [ ]:
grader.check("gradient_descent")

# IK using a jacobian
You can do either the Jacobian numerically or from matrices (extra credit) or both. 
The routines you need to edit are in arm_ik_jacobian.py

## Practice Jacobian
This is a 1d, stand-alone Jacobian. 

In [ ]:
import arm_ik_jacobian as ik_jacobian

In [ ]:
# Do this first to make sure you know how to calculate a Jacobian for 1 link
#  There's additional prints that print out the angle, moved point, etc.
print("Checking practice jacobian")
ang = ik_jacobian.practice_jacobian()
exp_ang = -0.032
if not np.isclose(ang, exp_ang, atol=0.01):
    print(f"Practice jacobian: Expected {exp_ang}, got {ang}")
else:
    print(f"Passed practice jacobian test")

In [ ]:
grader.check("practice_jacobian")

## Calculate jacobian
You need to do one of calculate_jacobian_numerically or calculate_jacobian (from matrices). Doing both will win you prizes and acclaim (and extra credit)

_Type your answer here, replacing this text._

In [ ]:
# Create the arm geometry
base_size_param = (1.0, 0.5)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)

arm_geometry = afk.create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

# Set some initial angles
angles_check = [0.0, np.pi/4, 0, [-np.pi/4.0, np.pi/4.0, -np.pi/4.0]]
afk.set_angles_of_arm_geometry(arm_geometry, angles_check)

# The location to reach for
target = (0.5, 1.5)

In [ ]:
### Numerical calculation check

In [ ]:
# First check - calculating the jacobian (2 x 4 matrix) calculated numerically
jacob_numerical = ik_jacobian.calculate_jacobian_numerically(arm_geometry, angles_check)
exp_jacob = np.array([[-0.9098, -0.4098, -0.1977, -0.0562], [-0.3535, -0.3535, -0.1414, 0.0]])
if not np.all(np.isclose(jacob_numerical, exp_jacob, atol=0.01)):
    print(f"Expected jacob\n{exp_jacob}, got\n{jacob}\n{np.isclose(jacob, exp_jacob, atol=0.01)}")
else:
    print("Passed numerical jacob test")

In [ ]:
### Matrix calculation check

In [ ]:
# Second check (optional) - calculating the jacobian using matrices (2 x 4 matrix)
jacob_matrix = ik_jacobian.calculate_jacobian(arm_geometry, angles_check)
if not np.all(np.isclose(jacob_matrix, exp_jacob, atol=0.01)):
    print(f"Expected jacob\n{exp_jacob}, got\n{jacob}\n{np.isclose(jacob, exp_jacob, atol=0.01)}")
else:
    print("Passed matrix jacob test")

In [ ]:
grader.check("jacobian")

## Do the pseudo inverse
Do the pseudo inverse (solve_jacobian) to solve for a set of angle changes given a set of x,y changes

In [ ]:
# Third check - pseudo inverse 1 x 5 matrix of angle changes needed to get the desired x,y change
delta_angles = ik_jacobian.solve_jacobian(np.array(exp_jacob), np.array([0.5, -0.2]))
exp_delta_angles = np.array([-1.456, 1.819,  0.506, -0.368])
if not np.all(np.isclose(delta_angles, exp_delta_angles, atol=0.01)):
    print(f"Expected delta angles\n{exp_delta_angles}, got\n{delta_angles}")
else:
    print("Passed solve jacobian test")

In [ ]:
grader.check("solve_jacobian")

## Use the Jacobian to move the gripper closer to the goal
Edit the code in jacobian_descent (which is very similar to gradient_descent). The checks *should* work with either the numerical or matrix jacobian.

_Type your answer here, replacing this text._

In [ ]:
# Main check - do we get out the new angles? Note, this assumes that you haven't changed up the step size
b_succ, angles_new, count = ik_jacobian.jacobian_follow_path(arm_geometry, angles_check, target, b_one_step=True)
ang_exp = [-0.0864, 0.8506, 0.01585, [-0.8024, 0, 0]]
if not len(angles_new) == 4:
    print(f"Expected {ang_exp}, got {angles_new}")
for a1, a2 in zip(angles_new, ang_exp):
    if not np.all(np.isclose(a1, a2, atol=0.01)):
        print(f"Expected {a1} got {a2}")
if not b_succ:
    print(f"Expected successful/improvement, got none")

In [ ]:
b_succ, angles_new, count = ik_jacobian.jacobian_follow_path(arm_geometry, angles_check, target, b_one_step=False)
afk.set_angles_of_arm_geometry(arm_geometry, angles_new)
dist = ik_gradient.distance_to_goal(arm_geometry, target)
if not b_succ:
    print(f"Expected successful/improvement, got none")
elif not np.isclose(dist, 0.062, atol=0.01):
    print(f"Expected distance to be close to 0.063, got {dist}, count {count}")
else: 
    print(f"Passed full solve check, dist {dist:0.2}")

In [ ]:
# Plot so you can see what it should look like
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

for i, a in enumerate([angles_check, angles_new]):
    # Actually get the matrices
    afk.set_angles_of_arm_geometry(arm_geometry, a)
    matrices = afk.get_matrices_all_links(arm_geometry)
    afk.plot_complete_arm(axs[i], arm_geometry, matrices)

    # Plot target too
    axs[i].plot(target[0], target[1], '+r')

axs[0].set_title("Before Jacobian descent")
axs[1].set_title(f"After Jacobian descent count {count} dist {dist:0.2}")

In [ ]:
grader.check("jacobian_descent")

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

# for all row, column in all_indices_from_where
#.   if this is the column for wrist torque 
#.      print(f"Row: {r}, Time step: {c // n_time_steps} Successful y/n: {pick_data[r, -1] == 1}, value: {pick_data[r, c]}")

In [ ]:
grader.check("hours_collaborators")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit through gradescope, inverse kinematics

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()